# 构建您的第一个活动

本笔记本将向您介绍根据从 Movielens 数据集收集的数据构建电影推荐模型的步骤。目标是根据特定用户推荐相关电影。

这些数据来源于 MovieLens 项目，您可以在以下单元格中的任何等待期间执行 Web 搜索，了解有关数据和潜在用途的更多信息。

## 如何使用笔记本

代码被分解为如下所示的单元格。在本页顶部有一个三角形 `Run` 按钮，您可以单击该按钮以执行每个单元格，并移至下一个单元格；也可以按 `Shift` + `Enter` 组合键，在单元格中执行并移至下一个单元格。

当一个单元格执行时，您会注意到该单元格运行时旁边有一行显示 `*`，或者它会更新为一个数字，以指示在执行完单元格中的所有代码后完成执行的最后一个单元格。


只需按照以下说明执行单元格，即可开始使用 Amazon Personalize。

## 导入 

Python 附带有许多库，我们需要导入这些库以及已安装的库来帮助我们，比如 [boto3](https://aws.amazon.com/sdk-for-python/)（适用于 Python 的 AWS SDK）和 [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/)，它们都是核心数据科学工具。

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
!conda install -y -c conda-forge unzip

接下来，您需要验证您的环境是否可以成功地与 Amazon Personalize 进行通信，下面的行就可以实现这一目的。

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## 配置数据

数据通过 Amazon S3 导入 Amazon Personalize，下面我们将指定您在 AWS 中为此练习创建的存储桶。

在下面，您将更新 `bucket` 变量，设置为您之前在 CloudFormation 步骤中创建的值，该值应位于您之前使用的文本文件中。`filename` 不需要更改。

### 指定存储桶和数据输出位置
如果您在 CloudFormation 模板部署期间自定义了 `bucket` 值，则确保更新该值。单击以下单元格进行更改。

In [ ]:
bucket = "personalizedemofirstnamelastname"       # replace with the name of your S3 bucket
filename = "movie-lens-100k.csv"

### 下载、准备和上载训练数据

目前您尚未在本地加载 MovieLens 数据进行检查，请执行以下行，下载最新副本并快速检查。

#### 下载并浏览数据集

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

#### 准备和上载数据

正如您所看到的，数据包含 UserID、ItemID、Rating 和 Timestamp。

我们现在将删除排名较低的项目，并删除 Rating 列，然后再为您构建模型。

完成后，现在我们将文件另存为新 CSV，然后上载到 S3。

通过执行以下单元格中的行来完成所有这些操作。

In [ ]:
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### 创建架构

Personalize 如何理解您的数据的核心组件来自下面定义的模式。此配置指导服务如何消化通过 CSV 文件提供的数据。注意列和类型与您在上面创建的文件中的内容对齐。

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### 创建并等待数据集组

Personalize 中最大的分组是数据集组，这将隔离您的数据、事件跟踪器、解决方案和活动。把共享通用数据集合的内容分组在一起。您可以随意更改下面的名称。

#### 创建数据集组

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-launch-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### 等待数据集组状态变为 ACTIVE

在我们可以在以下任何项目中使用数据集组之前，数据集组必须处于活跃状态，请执行下面的单元格并等待项目显示为活跃状态。

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### 创建数据集

分组之后，接下来要创建的是实际数据集，在此示例中，我们将仅为交互数据创建 1。执行以下单元格以创建数据集。

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-launch-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

#### 将策略附加到 S3 存储桶

Amazon Personalize 需要能够读取您之前创建的 S3 存储桶的内容。以下行将执行此操作。

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

#### 创建 Personalize 角色

此外，Amazon Personalize 需要能够担任 AWS 中的角色，以便拥有执行某些任务的权限，下面的行授予了此权限。

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## 导入数据

您之前创建了数据集组和数据集来存储您的信息，现在您将执行一个导入作业，从 S3 将数据加载到 Amazon Personalize 中，用于构建您的模型。

#### 创建数据集导入作业

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import1",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### 等待数据集导入作业状态变为 ACTIVE

导入作业可能需要一会儿才能完成，请等待直至看到它在下面处于活跃状态。

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 创建解决方案和版本

在 Amazon Personalize 中，经过训练的模型称为“解决方案”。模型经过训练后，每个解决方案都可以有许多与给定数据量相关的特定版本。

首先，我们将列出受支持的所有配方，配方是一种尚未接受过数据训练的算法。列出后，您将选择一个配方并使用它来构建您的模型。

### 选择配方

In [ ]:
list_recipes_response = personalize.list_recipes()
list_recipes_response

#### 用户个性化
[User-Personalization](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html)（aws-user-personalization）配方针对所有 USER_PERSONALIZATION 推荐方案进行了优化。在推荐项目时，它使用自动项目探索。

借助自动探索功能，Amazon Personalize 自动测试不同的项目推荐，了解用户如何与这些推荐项目交互，并改进可提高参与度和转化率的项目的推荐。当您的目录快速变化，或者当新项目（如新闻文章或促销）与新用户更相关时，这可改进项目发现和参与度。

您可以平衡探索的量（推荐更频繁地探索较低交互数据或相关性较低的项目）与要利用的量（推荐基于我们的了解内容或相关性）。Amazon Personalize 根据隐式用户反馈自动调整未来的推荐。

首先，通过在上面的配方列表中找到 ARN 来选择配方。

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### 创建并等待解决方案

首先，您将使用 API 创建解决方案，然后创建一个版本。系统将需要几分钟时间来训练模型并创建解决方案的版本。开始后，您会收到通知告知您操作已在进行中，这样您就可以休息一下了，比如喝杯咖啡等。

#### 创建解决方案

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-user-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### 创建解决方案版本

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### 等待解决方案版本状态变为 ACTIVE

这大约需要 40-50 分钟。

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### 获取解决方案版本的指标

既然您的解决方案和版本已经存在，您就可以获取指标来判断性能。这些指标不是特别好，因为它是一组演示数据，但如果是更大更复杂的数据集，您应该会看到改进。

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

我们建议阅读[文档](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)了解指标，但为了方便起见，我们还复制了以下文档的部分内容。

您需要了解 Personalize 中有关评估的以下术语：

– *相关推荐*是指与特定用户测试数据中的值相匹配的推荐。
– *排名*是指推荐清单中推荐项目的位置。位置 1（列表顶部）假定与用户最相关。
– *查询*是指 GetRecommendations 调用的内部等效项。

Personalize 生成的指标包括：

– 覆盖范围：训练数据（包括项目和交互数据集）中来自所有查询的唯一推荐项目数占唯一项目总数的比例。
– mean_reciprocal_rank_at_25：对所有查询提出的前 25 项推荐中，第一项相关推荐的[倒数排名的平均值](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)。如果您对单个排名最高的推荐感兴趣，则此指标是合适的。
– normalized_discounted_cumulative_gain_at_K：折损增益假设推荐列表中较下面的推荐的相关性低于较上面的推荐。因此，每项推荐都被一个因数折损（为推荐赋予较低的权重），具体取决于推荐的位置。为了产生 K 处的[累积折损增益](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)（DCG），将前 K 个推荐中的每个相关折损推荐加在一起。归一化折损累积增益（NDCG）是 DCG 除以理想 DCG，使得 NDCG 在 0-1 之间。（理想 DCG 是按相关性对前 K 个推荐进行排序。） Amazon Personalize 使用 1/log(1 + position) 权重因子，其中列表顶部为位置 1。此指标奖励出现在列表顶部附近的相关项目，因为列表的顶部通常会更受关注。
– precision_at_K：前 K 个推荐中相关推荐的数量除以 K。此指标奖励相关项目的精确推荐。

## 创建并等待活动

现在您有了一个有效的解决方案版本，您需要创建一个活动，以便用于您的应用程序。活动是托管解决方案版本；可以查询推荐的端点。通过估计吞吐量（用户每秒的个性化请求）来设置定价。部署活动时，您可以设置每秒事务数（TPS）最小值（`minProvisionedTPS`）。与 AWS 中的许多服务一样，此服务将根据需求自动扩展，但如果延迟至关重要，您可能希望提前预置以满足更大的需求。对于此演示，最小吞吐量阈值设置为 1。有关更多信息，请参阅[定价](https://aws.amazon.com/personalize/pricing/)页面。

如上所述，我们的解决方案使用的 user-personalization 配方支持自动浏览"冷"项目。您可以控制在创建活动时执行的浏览量。`itemExplorationConfig` 数据类型支持 `explorationWeight` 和 `explorationItemAgeCutOff` 参数。浏览权重决定推荐包含交互较少数据或相关性较低项目的频率。值越接近 1.0，则浏览量越多。在为零时，不会进行浏览，推荐基于当前数据（相关性）。浏览项目期限截止日期根据自上次交互以来的时间范围确定要浏览的项目。提供自上次交互以来的最大项目期限（以天为单位），以定义项目浏览的范围。值越大，在浏览过程中考虑的项目越多。对于以下活动，我们将指定 0.5 的浏览权重。

#### 创建活动

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-demo-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.5"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### 等待活动状态变为 ACTIVE

这大约需要 10 分钟。

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 获取样本推荐

活动状态变为活跃后，您就可以获得推荐。首先，我们需要从集合中选择一个随机用户。然后，我们将创建一些辅助函数，以获取电影信息以显示推荐，而不是仅显示 ID。

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE']


#### 调用 GetRecommendations

使用您在上面获得的用户，以下行将为您提供推荐，并返回推荐的电影列表。


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['OriginalRecs'])
recommendations_df

## 审核

使用上述代码，您已成功训练了一个深度学习模型，可根据之前的用户行为生成电影推荐。思考包含这些数据的其他类型的问题，以及构建这样的系统提供这些推荐可能会是什么样。

现在，您已准备好进入下一个笔记本 `2.View_Campaign_And_Interactions.ipynb`



## 适用于下一个笔记本的注意事项：

下一个笔记本需要一些值，请执行以下单元格来存储这些值，以便将它们复制并粘贴到练习的下一部分。

In [ ]:
%store campaign_arn

In [ ]:
%store dataset_group_arn

In [ ]:
%store solution_version_arn

In [ ]:
%store solution_arn

In [ ]:
%store dataset_arn

In [ ]:
%store campaign_arn

In [ ]:
%store schema_arn

In [ ]:
%store bucket

In [ ]:
%store filename

In [ ]:
%store role_name

In [ ]:
%store recommendations_df

In [ ]:
%store user_id